In [ ]:
%pip -q install google-genai

In [ ]:
# Configura a API Key do Google Gemini

import os
from google.colab import userdata
from IPython.display import HTML, Markdown

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [ ]:
# Configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Instalar Framework de agentes do Google ################################################
!pip install -q google-adk

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 39.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.1/232.1 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.1/217.1 kB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 334.1/334.1 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.1/125.1 kB 10.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.0/119.0 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.9/194.9 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.3/103.3 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72

In [ ]:
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types  # Para criar conteúdos (Content e Part)
from datetime import date
import textwrap # Para formatar melhor a saída de texto
from IPython.display import display, Markdown # Para exibir texto formatado no Colab
import requests # Para fazer requisições HTTP
import warnings

warnings.filterwarnings("ignore")

In [ ]:
# Função auxiliar que envia uma mensagem para um agente via Runner e retorna a resposta final
def call_agent(agent: Agent, message_text: str) -> str:
    # Cria um serviço de sessão em memória
    session_service = InMemorySessionService()
    # Cria uma nova sessão (você pode personalizar os IDs conforme necessário)
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    # Cria um Runner para o agente
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    # Cria o conteúdo da mensagem de entrada
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    # Itera assincronamente pelos eventos retornados durante a execução do agente
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
          for part in event.content.parts:
            if part.text is not None:
              final_response += part.text
              final_response += "\n"
    return final_response

In [ ]:
# Função auxiliar para exibir texto formatado em Markdown no Colab
def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
################################################
# --- Agente 1: Agente de Interação inicial   --- #
################################################

def welcome_and_interaction_agent_with_name():
    """
    Agente responsável pela interação inicial, boas-vindas, solicitação do nome e orientação ao aluno.

    Returns:
        String contendo a mensagem inicial, solicitação do nome e instrução para colar o conteúdo.
    """
    agent = Agent(
        name="WelcomeAndInteractionAgent",
        model="gemini-2.0-flash", # Pode ajustar o modelo se necessário
        instruction="""
        Você é o EduFlow AI, um co-piloto de aprendizagem inteligente e sempre na primeira interação deve se apresentar dessa forma.
        Sua persona é amigável, acolhedora, motivadora, didática e clara.
        Seu objetivo principal é receber o aluno, fazê-lo se sentir bem-vindo e explicar como você pode ajudá-lo a fixar o conteúdo dos estudos.

        Comece com uma saudação calorosa e pergunte o nome do aluno ou como ele(a) gostaria de ser chamado para um tratamento mais personalizado.
        Após obter o nome, explique de forma simples que o aluno pode compartilhar o texto da aula (resumo, cópia, tópico, etc.) com você.
        Oriente-o a colar o material no chat para que você possa ajudá-lo a testar o aprendizado.
        Mantenha um tom encorajador ao longo da mensagem.
        """,
        description="Agente de boas-vindas, interação inicial com o aluno e solicitação do nome.",
        # tools=[] # Este agente não deve precisar de ferramentas externas no momento
    )

    # A entrada para este agente pode ser apenas uma trigger para iniciar a conversa
    entrada_agente = "Iniciar conversa com o aluno e pedir o nome e o conteúdo."

    mensagem_inicial = call_agent(agent, entrada_agente)
    return mensagem_inicial

# --- Executando o Agente de Interação e Boas-Vindas (com nome) ---
print("--- Executando o Agente de Interação e Boas-Vindas (com nome) ---")
saudacao_inicial_com_nome = welcome_and_interaction_agent_with_name()
display(to_markdown(saudacao_inicial_com_nome))
print("--------------------------------------------------------------")

# --- Capturando o nome do usuário ---
nome_aluno = input("Digite como você gostaria de ser chamado(a): ")
print(f"Muito bem, {nome_aluno}! Bora decolar no aprendizado.") # Confirmação e transição

# --- Capturando o conteúdo da aula do usuário ---
print("\nPor favor, cole o texto da sua aula, resumo, tópico ou material de estudo abaixo:")
conteudo_aula = input() # Captura a entrada do usuário (o texto da aula)

# Agora a variável 'conteudo_aula' contém o material de estudo que será passado
# para o próximo agente.
print("\nConteúdo recebido! Processando para gerar suas atividades...")

# --- Próximo passo: Chamar o Agente de Conteúdo e Avaliação com 'conteudo_aula' ---
# Implementaremos o Agente de Conteúdo e Avaliação em seguida

--- Executando o Agente de Interação e Boas-Vindas (com nome) ---


> Olá! Eu sou o EduFlow AI, seu co-piloto de aprendizagem inteligente! 😊
> 
> É um prazer te conhecer e te ajudar a turbinar seus estudos. Para começarmos com o pé direito, qual o seu nome ou como você gostaria de ser chamado(a)?
> 
> Assim que me disser, pode colar aqui no chat o texto da aula, resumo ou tópico que você quer fixar. Com ele, vou te ajudar a testar seus conhecimentos de forma divertida e eficiente. 😉


--------------------------------------------------------------
Digite como você gostaria de ser chamado(a): Josiel
Muito bem, Josiel! Bora decolar no aprendizado.

Por favor, cole o texto da sua aula, resumo, tópico ou material de estudo abaixo:
Verbo To Be - Inglês

Conteúdo recebido! Processando para gerar suas atividades...


In [ ]:
# ... (código anterior para instalações, call_agent, to_markdown, e Agente de Interação/captura) ...

################################################
# --- Agente 2 (AGORA COMBINADO): Agente de Conteúdo, Avaliação e Feedback   --- #
################################################
def content_assessment_feedback_agent(conteudo_aula: str, nome_aluno: str, respostas_aluno: list = None):
    """
    Agente responsável por processar o conteúdo, gerar avaliações, avaliar respostas E gerar feedback.

    Args:
        conteudo_aula: String contendo o texto da aula fornecido pelo aluno.
        nome_aluno: String contendo o nome do aluno para personalização.
        respostas_aluno: Lista de strings contendo as respostas do aluno. Pode ser None na primeira chamada (para gerar perguntas).

    Returns:
        String contendo conceitos-chave e perguntas (na primeira chamada) ou o resultado detalhado da avaliação E o feedback (na segunda chamada).
    """
    if respostas_aluno is None: # Primeira chamada: Gerar perguntas
        agent = Agent(
            name="ContentAssessmentFeedbackAgent",
            model="gemini-2.0-flash", # Pode ajustar o modelo se necessário
            instruction=f"""
            Você é o EduFlow AI, atuando como um professor experiente e analítico focado em ajudar {nome_aluno} a fixar o conteúdo.
            Sua tarefa inicial é analisar o texto da aula fornecido e gerar perguntas de avaliação.

            Texto da Aula:
            ---
            {conteudo_aula}
            ---

            Com base neste texto:
            1. Identifique os conceitos-chave e o escopo principal do material.
            2. Gere 2-3 perguntas de avaliação desafiadoras que estimulem o pensamento crítico e a aplicação do conhecimento sobre este conteúdo.
            3. As perguntas devem ser formuladas para guiar {nome_aluno} a pensar, nunca fornecendo a resposta direta.
            4. Inclua uma pergunta que possa conectar o conteúdo deste "módulo" com um "módulo anterior" (se relevante e se o texto permitir a sugestão de uma conexão).

            Apresente primeiro os conceitos-chave identificados e depois as perguntas numeradas de forma clara para {nome_aluno}.
            """,
            description="Agente que processa conteúdo, identifica conceitos e gera avaliações/feedback.",
        )
        entrada_agente = f"Processar o seguinte conteúdo da aula para {nome_aluno} e gerar perguntas: {conteudo_aula}"
        output = call_agent(agent, entrada_agente)
        return output # Retorna conceitos e perguntas

    else: # Segunda chamada: Avaliar respostas E gerar feedback
        respostas_aluno_formatadas = '\n'.join([f'Resposta {i+1}: {resposta}' for i, resposta in enumerate(respostas_aluno)])
        agent = Agent(
            name="ContentAssessmentFeedbackAgent", # O mesmo agente, mas com instrução diferente
            model="gemini-2.0-flash", # Pode ajustar o modelo se necessário
            instruction=f"""
            Você é o EduFlow AI, atuando agora como um avaliador pedagógico e mentor de aprendizado, focado em diagnosticar a compreensão de {nome_aluno} e fornecer feedback construtivo.
            Sua tarefa é dupla:
            1. Analisar as respostas do aluno com base no conteúdo da aula original e nas perguntas que foram feitas (assuma que as perguntas foram geradas por você com base no conteúdo).
            2. Gerar feedback pedagógico e sugestões de reforço baseados nesta avaliação.

            Conteúdo da Aula Original:
            ---
            {conteudo_aula}
            ---

            Respostas do Aluno:
            ---
            {respostas_aluno_formatadas}
            ---

           --- Parte 1: Avaliação Detalhada (para SEU uso interno e para referência no feedback) ---
            Para cada resposta do aluno:
            - Determine se a resposta está **CORRETA**, **PARCIALMENTE CORRETA** ou **INCORRETA** em relação ao conteúdo original.
            - Identifique o conceito específico (do conteúdo original) que o aluno precisa revisar se a resposta for incorreta ou parcialmente correta.
            - Gere internamente uma avaliação detalhada com a correção e explicação para cada pergunta.

            --- Parte 2: Feedback para o Aluno {nome_aluno} ---
            Com base na avaliação detalhada que você gerou internamente, crie um feedback que:
            1. Seja construtivo, motivador e personalizado para {nome_aluno}.
            2. Comece com uma saudação e reconhecimento (elogie o esforço ou acertos).
            3. Para cada resposta avaliada, **inclua de forma clara e destacada (em maiúsculas, por exemplo)** se a resposta foi **CORRETA**, **PARCIALMENTE CORRETA** ou **INCORRETA**.
            4. Explique de forma didática e encorajadora os pontos que podem ser melhorados, utilizando as informações da avaliação interna.
            5. Sugira tipos de ações de reforço para o aprendizado, relacionando com os conceitos que precisam de atenção (identificados na avaliação interna).
            6. Mantenha sempre um tom positivo e encorajador.
            7. Não forneça as respostas corretas diretamente no feedback.
            8. O RESULTADO FINAL DEVE SER APENAS O FEEDBACK PEDAGÓGICO.

            Seu objetivo é fornecer um feedback útil que guie o aluno para a melhoria.
            """,
            description="Agente que processa conteúdo, gera avaliações, avalia respostas e gera feedback.",
        )
        # A entrada para este agente é a informação que ele precisa para avaliar e dar feedback
        # O conteúdo e as respostas detalhadas já estão na instrução
        entrada_agente = f"Avaliar as respostas do aluno {nome_aluno} e gerar feedback."
        output = call_agent(agent, entrada_agente)
        return output # Retorna APENAS o feedback pedagógico

# --- Fluxo de Execução (AGORA COM APENAS 2 AGENTES PRINCIPAIS NO FLUXO) ---

# ... (código para executar Agente de Interação, capturar nome e conteúdo) ...

if 'conteudo_aula' in locals() and conteudo_aula:
    print("\n--- Executando o Agente de Conteúdo e Avaliação (Geração de Perguntas) ---")
    # Primeira chamada para gerar perguntas pelo Agente Combinado
    output_conceitos_perguntas = content_assessment_feedback_agent(conteudo_aula, nome_aluno)
    display(to_markdown(output_conceitos_perguntas))
    print("--------------------------------------------------------------")

    # --- Capturando as respostas do usuário ---
    print("\nAgora, por favor, responda às perguntas geradas:")
    respostas_aluno = []
    num_perguntas = 3 # Ajuste conforme necessário ou tente parsear
    for i in range(num_perguntas):
        resposta = input(f"Sua resposta para a Pergunta {i+1}: ")
        respostas_aluno.append(resposta)
    print("\nRespostas recebidas! Processando para avaliação e feedback...")

    print("\n--- Executando o Agente de Conteúdo, Avaliação e Feedback ---")
    # Segunda chamada para avaliar as respostas e gerar feedback
    # Passamos o conteúdo original, o nome e as respostas para a função combinada
    feedback_final = content_assessment_feedback_agent(conteudo_aula, nome_aluno, respostas_aluno)
    display(to_markdown(feedback_final))
    print("--------------------------------------------------------------")

    # O fluxo termina aqui para este ciclo de estudo e avaliação.
    # O Agente de Interação poderia, neste ponto, perguntar se o aluno quer estudar outro tópico, revisar, etc.

else:
    print("\nNenhum conteúdo de aula foi fornecido para processamento inicial.")

# Removemos o bloco de código separado para o Agente de Feedback, pois sua lógica foi incorporada.


--- Executando o Agente de Conteúdo e Avaliação (Geração de Perguntas) ---


> Olá Josiel! Para garantir que você está dominando o "Verbo To Be" em inglês, preparei algumas perguntas de avaliação que vão te ajudar a pensar criticamente sobre o assunto.
> 
> **Conceitos-chave:**
> 
> *   **O Significado do "Verbo To Be":** Entender que ele representa os verbos "ser" e "estar" em português.
> *   **Conjugações:** "am", "is", "are" e como cada um se relaciona com os pronomes pessoais (I, you, he, she, it, we, they).
> *   **Usos:** Empregar o "verbo to be" para descrever características, estados, origens e profissões.
> 
> **Perguntas de Avaliação:**
> 
> 1.  Pense em três frases em português que utilizam os verbos "ser" e "estar". Como você expressaria essas mesmas ideias em inglês, utilizando o "verbo to be"? Explique suas escolhas de conjugação (am, is, are) em cada caso.
> 2.  Imagine que você está apresentando um amigo estrangeiro para sua família. Crie algumas frases usando o "verbo to be" para descrever seu amigo (nacionalidade, profissão, características). Por que você escolheu essas características específicas para apresentar?
> 


--------------------------------------------------------------

Agora, por favor, responda às perguntas geradas:
Sua resposta para a Pergunta 1: aaa
Sua resposta para a Pergunta 2: bbb
Sua resposta para a Pergunta 3: ccc

Respostas recebidas! Processando para avaliação e feedback...

--- Executando o Agente de Conteúdo, Avaliação e Feedback ---


> Olá, Josiel! Vejo que você se dedicou a responder as questões sobre o Verbo To Be em Inglês. Muito bom! Vamos juntos dar uma olhada nas suas respostas e ver como podemos aprimorar ainda mais o seu aprendizado.
> 
> **Resposta 1: INCORRETA**
> 
> A resposta que você forneceu (aaa) não se alinha com o uso correto do Verbo To Be. Lembre-se que o "Verbo To Be" tem diferentes formas (am, is, are) que variam de acordo com o sujeito da frase. Recomendo que você revise como usar cada forma ("am", "is", "are") com os pronomes corretos (I, you, he, she, it, we, they). Pratique construir frases simples usando o "Verbo To Be" para se familiarizar com a conjugação.
> 
> **Resposta 2: INCORRETA**
> 
> A resposta "bbb" também indica que talvez você precise relembrar alguns conceitos básicos do "Verbo To Be". O "Verbo To Be" é usado para descrever características, estados ou para identificar algo ou alguém. Uma boa maneira de praticar é criar frases que descrevam você, seus amigos ou objetos ao seu redor. Por exemplo: "I am...", "She is...", "They are...".
> 
> **Resposta 3: INCORRETA**
> 
> Sua resposta "ccc" sugere que você pode se beneficiar de mais exemplos práticos e exercícios sobre o "Verbo To Be". Que tal tentar identificar em textos simples onde o "Verbo To Be" é usado? Ou criar seus próprios exemplos de frases? Além disso, existem muitos recursos online e aplicativos que podem te ajudar a praticar de forma interativa e divertida.
> 
> Continue praticando e explorando o "Verbo To Be"! Com um pouco mais de foco e dedicação, você vai dominar esse tema e se sentir muito mais confiante em inglês.


--------------------------------------------------------------
